In [1]:
#Import libraries
import pandas as pd
import numpy as np

In [49]:
#Load budget data
budget_file = 'plano_orcamentario_2018.txt'
data = pd.read_csv(budget_file, sep=';')

# 1. Initial Data Treatment

In [52]:
#Rename columns
new_cols = {
    'codigo': 'code',
    'descricao': 'description',
    'codUnidadeOrcamentaria': 'budget_unit_code',
    'codigoFuncao': 'function_code',
    'codigoSubFuncao': 'subfunction_code',
    'codigoPrograma': 'program_code',
    'codigoAcao': 'action_code',
    'codPOIdAcompanhamento':  'managing_code',
    'descPOIdAcompanhamento': 'managing_description',
    'numAno': 'year'
}

data.rename(columns=new_cols, inplace=True)

# 2. Early Exploration

## 2.1. Unique Field Values

In [54]:
#Description of fields
def check_alphanumeric(series):
    is_digit = max([max([ch.isdigit() for ch in str(val)]) for val in series])
    is_alpha = max([max([ch.isalpha() for ch in str(val)]) for val in series])
    
    if is_digit and is_alpha:
        return 'alphanumeric'
    elif is_digit:
        return 'numbers only'
    elif is_alpha:
        return 'letters only'


description = {}
for col in data.columns:
    description.update({
        col: [data[col].nunique(), check_alphanumeric(data[col])]
    })
    
description = pd.DataFrame(description).T
description.columns = ['Unique Vals', 'Data Type']
description

,Unique Vals,Data Type
id,15465,numbers only
code,150,alphanumeric
description,5059,alphanumeric
budget_unit_code,444,numbers only
function_code,29,numbers only
subfunction_code,100,numbers only
program_code,137,numbers only
action_code,2746,alphanumeric
managing_code,54,alphanumeric
managing_description,33,alphanumeric


* We have fields with very different amounts of unique values;
* Our next step should be to understand how these fields relate to each other. That means, at this point, to check if there is any sort of relationship between the values of 2 or more fields that could helps normalize this dataset.

## 2.2. Relationship Between Fields

In [66]:
def check_relationship(data, first_col, second_col):
    df_first = data.groupby(by=first_col).agg({second_col: 'nunique'})
    df_second = df = data.groupby(by=second_col).agg({first_col: 'nunique'})
    
    max_first = df_first.max()[0]
    max_second = df_second.max()[0]
    
    return '{} {} \t: {} {}'.format(
        1 if max_second == 1 else 'N',
        first_col,
        1 if max_first == 1 else 'M',
        second_col
    )

In [67]:
#Check possible relationships
cols_to_check = data.columns[1:-1]
for i, col in enumerate(cols_to_check[:-1]):
    next_col = cols_to_check[i+1]
    print(check_relationship(data, col, next_col))

N code 	: M description
N description 	: M budget_unit_code
N budget_unit_code 	: M function_code
N function_code 	: M subfunction_code
N subfunction_code 	: M program_code
N program_code 	: M action_code
N action_code 	: M managing_code
N managing_code 	: 1 managing_description


Apparently theres only one relationship we can more easily leverage at this point, the one between the "managing_code" and "managing_description" fields. For this one, we see that there is only a single description for each code but, on the other way around, the same description can be applied for several different codes. Since this does not seem to make a lot of sense, let's take a deeper look at it.

In [68]:
#Check for the "managing_description" values for different "managing_code"
aux = data.groupby(by='managing_description').agg({'managing_code': 'nunique'})
aux = aux[aux['managing_code'] > 1]

aux

,managing_code
managing_description,
Sem informação,22


There is only a single description being used in more than one code at the same time. Looking at this description, it means "No information". So it is not that the same description is being used for 22 different codes, but that these codes do not have any related description.